In [ ]:
import nest_asyncio
nest_asyncio.apply()

from ib_insync import util
util.startLoop()

In [ ]:
# Simulate a big qualifyAsync
from ib_insync import IB, Option, util
import pandas as pd
import yaml
import asyncio
import random
import logging

MARKET = 'snp'
root_path = "C:/Users/User/Documents/Business/Projects/asyncib/"
data_path = root_path+'data/'+MARKET.lower()+'/'

# get the connection IDs
with open(root_path+'var.yml') as f:
    data=yaml.safe_load(f)
    
HOST = data["COMMON"]["HOST"]
PORT = data[MARKET.upper()]["PORT"]
CID = data["COMMON"]["CID"]

# make the contract chains
df_chains = pd.read_pickle(data_path+'df_chains.pkl')

puts = [Option(symbol=s, lastTradeDateOrContractMonth=e, strike=k, right='P', exchange=x) 
                for s, e, k, x in 
                zip(df_chains.symbol, df_chains.expiry, df_chains.strike, ['NSE' 
                    if MARKET.upper() == 'NSE' else 'SMART']*len(df_chains))]

calls = [Option(symbol=s, lastTradeDateOrContractMonth=e, strike=k, right='C', exchange=x) 
                for s, e, k, x in 
                zip(df_chains.symbol, df_chains.expiry, df_chains.strike, ['NSE' 
                    if MARKET.upper() == 'NSE' else 'SMART']*len(df_chains))]

raw_cs = puts+calls

In [ ]:
contracts = random.sample(raw_cs, 200) # choose 15 random contracts
util.logToConsole(level=logging.FATAL)

with IB().connect(HOST, PORT, CID) as ib:
    loop = asyncio.get_running_loop()
    tasks = [asyncio.create_task(ib.qualifyContractsAsync(c)) for c in contracts]
    finished, unfinished = loop.run_until_complete(asyncio.wait(tasks, timeout=5, return_when=asyncio.ALL_COMPLETED))

[task.result() for task in finished]

In [ ]:
def qualCoro(ib, contracts):
    loop = asyncio.get_running_loop()
    tasks = [asyncio.create_task(ib.qualifyContractsAsync(c)) for c in contracts]
    print(tasks)

    print("Get first result")
    finished, unfinished = loop.run_until_complete(asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED))
    
    for task in finished:
        print(task.result())
    print("unfinished:", len(unfinished))

    return task.result()